## ABC Simple Installation System: Post-processing Upgrade Results - for Low Rise MF & SF
(savings relative to thermal loads) \
Created on: 10/12/2021 \
By: Lixi Liu (Lixi.Liu@nrel.gov)

Note: change kernel to that of EULP Calibration & Validation before running

In [6]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
print(f'Notebook path: {os.getcwd()}')
from pathlib import Path
import eulpcv.resstock_enduse_categories as res_enduse_categories

Notebook path: /Users/lliu2/Documents/GitHub/ResStock/files


### Download results online
* unprocessed upgrade results: S3/resbldg-datasets/abc-sis

### Initialize

In [7]:
# local path to downloaded results
pkg_version = f'packages LRMF + SF' # <----- N=6
result_dir = Path('/Users/lliu2/Documents/ABC Simple Intall Sys/results')
result_path = result_dir / pkg_version / 'results'
resdir = result_dir / pkg_version / 'processed results'

## create folder for post-processed results:
outdir = result_dir / pkg_version / 'result summary'
plot_path = outdir / 'plots'

os.makedirs(resdir, exist_ok=True)
os.makedirs(outdir, exist_ok=True)
os.makedirs(plot_path, exist_ok=True)

print(f'Run results come from: \n   {result_path}')
print(f'Output directory: \n   {outdir}')

Run results come from: 
   /Users/lliu2/Documents/ABC Simple Intall Sys/results/packages LRMF + SF/results
Output directory: 
   /Users/lliu2/Documents/ABC Simple Intall Sys/results/packages LRMF + SF/result summary


### Variables

In [16]:
upgrade_rename_dict = {
    "Airtight Envelope": "Env", #1
    "Extra-Airtight Envelope": "ExtraEnv", #2
    "Ultra-Airtight Envelope - HRV/ERV": "UltraEnv", #3
    "Airtight Envelope + MSHP + HPWH": "Env + MSHP + HPWH", #4
    "Extra-Airtight Envelope + MSHP + HPWH": "ExtraEnv + MSHP + HPWH", #5
    "Ultra-Airtight Envelope - HRV/ERV + MSHP + HPWH": "UltraEnv + MSHP + HPWH", #6
}
print('"upgrade_rename_dict" created')


## Get thermal load end uses
# electric
thermal_eu_main = ['hot_water', 'heating', 'cooling', 'hvac_fan_pump']
eu_dict = res_enduse_categories.enduse_category_dict()
eu_df = pd.DataFrame.from_dict(eu_dict, orient='index', columns=['enduse']) # turn eu_dict to df
thermal_eu_df = eu_df.query('enduse in @thermal_eu_main')
thermal_eu_df.index = 'simulation_output_report.' + thermal_eu_df.index
thermal_eu = thermal_eu_df.index

# gas
thermal_eu_gas = ["natural_gas_heating_therm",
                  "natural_gas_water_systems_therm",]
thermal_eu_gas = list('simulation_output_report.' + x for x in thermal_eu_gas)

# others
thermal_eu_other = ["fuel_oil_heating_mbtu",
                    "fuel_oil_water_systems_mbtu",
                    "propane_heating_mbtu",
                    "propane_water_systems_mbtu",
                    "wood_heating_mbtu",]
thermal_eu_other = list('simulation_output_report.' + x for x in thermal_eu_other)

print('"thermal_eu(s)" defined')


# conversion multipliers
kwh_to_mmbtu = 0.003412141633127942
therm_to_mmbtu = 0.1

print('"conversion factors" defined')

"upgrade_rename_dict" created
"thermal_eu(s)" defined
"conversion factors" defined


### Functions

In [9]:
def add_thermal_load_sqft_eui(df, ref):
    """
    ARG:
        ref (df): baseline df
    RETURN:
        df with added cols: 'sqft', 'gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_mmbtupersqft'
    """
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df['sqft'] = res['simulation_output_report.floor_area_conditioned_ft_2']
    
    df['thermal_gas_therm'] = df[thermal_eu_gas].sum(axis=1)
    df['thermal_elec_kwh'] = df[thermal_eu].sum(axis=1)
    df['thermal_other_mmbtu'] = df[thermal_eu_other].sum(axis=1)
    df['thermal_site_mmbtu'] = df['thermal_elec_kwh']*kwh_to_mmbtu + df['thermal_gas_therm']*therm_to_mmbtu + df['thermal_other_mmbtu']
    
    df['gas_eui_thermpersqft'] = df['thermal_gas_therm'].divide(df['sqft']) # therm/sqft
    df['elec_eui_kwhpersqft'] = df['thermal_elec_kwh'].divide(df['sqft']) # kwh/sqft
    df['site_eui_mmbtupersqft'] = df['thermal_site_mmbtu'].divide(df['sqft']) # mmbtu/sqft
    
    for col in ['sqft','gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_mmbtupersqft']:
        df.loc[df['simulation_output_report.applicable']==False, col] = np.nan
    
    return df
print('func loaded: "add_thermal_load_sqft_eui"')


# # fix 1ACH50 upgrade costs in UEnv-Mech & UEnv-Rec
# pkg_to_fix = [
#     "Ultra Air Tight Envelope - Mec Vent", #2 (v01)
#     "Ultra Air Tight Envelope - Rec Vent", #3 (v01)
#     "Ultra Air Tight Envelope - Mech Vent + Roof Insulation", #5 (V01)
#     "Ultra Air Tight Envelope - Mech Vent + MSHP", #7 (v02, #2)
#     "Ultra Air Tight Envelope - Rec Vent + MSHP", #8 (v02, #3)
#     "Ultra Air Tight Envelope - Mech Vent + HPWH", #10 (v01)
#     "Ultra Air Tight Envelope - Rec Vent + MSHP + HPWH", #12 (v02, #5)
# ]

# # prev, all at 5.1 $/sqft
# air_seal_cost = {
#     "50 ACH50": 10.21,
#     "40 ACH50": 8.17,
#     "30 ACH50": 6.13,
# }

# def fix_upgrade_costs(df, ref):
#     print('  fixing 1 ACH50 air sealing cost...')
    
#     upcost_prev = df['simulation_output_report.upgrade_cost_usd'].copy()
#     res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    
#     res['air_seal_cost'] = res['build_existing_model.infiltration'].map(air_seal_cost)
#     res['air_seal_cost'] = res['air_seal_cost'].fillna(5.1)

#     df['simulation_output_report.upgrade_cost_usd'] += (res['air_seal_cost']-5.1)*\
#         res['simulation_output_report.floor_area_conditioned_ft_2']
#     df['simulation_output_report.upgrade_cost_usd']
    
#     upcost_delta = df['simulation_output_report.upgrade_cost_usd'] - upcost_prev
#     upcost_delta_avg = upcost_delta.mean()
#     upcost_change_count = upcost_delta[upcost_delta>0].count()
#     (df['simulation_output_report.upgrade_cost_usd'] - upcost_prev)
#     print(f'  1 ACH50 air sealing cost increased for {upcost_change_count} simulations, by: {upcost_delta_avg}')
    
#     return df
# print('func loaded: "fix_upgrade_costs"')


func loaded: "add_thermal_load_sqft_eui"


### 1. BASELINE results

In [10]:
res = pd.read_csv(result_path / 'results_up00.csv')

# (1) get sqft, gas/elec/site eui
res = add_thermal_load_sqft_eui(res, res)

# check
Nbldg = 488745 #N = 489k = 110k MF + 346k SFD + 32k SFA after downselect
Njob = 500 # <--

jobs_missing = set(range(1,Njob)) - set(res.job_id.unique())
print(f'- {len(jobs_missing)} jobs missing: {jobs_missing}')
print(f'- {Nbldg-len(res)} buildings ({((Nbldg-len(res))/Nbldg*100):.2f}%)')

res

- 0 jobs missing: set()
- 0 buildings (0.00%)


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,simulation_output_report.window_area_ft_2,simulation_output_report.wood_heating_mbtu,sqft,thermal_gas_therm,thermal_elec_kwh,thermal_other_mmbtu,thermal_site_mmbtu,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_mmbtupersqft
0,1,279,2021-12-05 20:45:09,2021-12-05 20:46:30,Success,NaN,NaN,NaN,"CBSA Seattle-Tacoma-Bellevue, WA",True,...,396.51,0.0,2176.0,1407.450612,1066.301104,0.0,144.383432,0.646806,0.490028,0.066353
1,2,37,2021-12-05 19:27:28,2021-12-05 19:28:12,Success,NaN,NaN,NaN,Non-CBSA West South Central,True,...,67.54,0.0,617.0,161.753257,2007.548864,0.0,23.025367,0.262161,3.253726,0.037318
2,3,251,2021-12-05 20:17:48,2021-12-05 20:18:54,Success,NaN,NaN,NaN,"CBSA New York-Newark-Jersey City, NY-NJ-PA",True,...,85.56,0.0,2176.0,731.062251,1573.735227,0.0,78.476033,0.335966,0.723224,0.036064
3,5,106,2021-12-05 19:34:35,2021-12-05 19:35:33,Success,NaN,NaN,NaN,"CBSA New York-Newark-Jersey City, NY-NJ-PA",True,...,9.79,0.0,333.0,87.397429,363.215323,0.0,9.979085,0.262455,1.090737,0.029967
4,6,87,2021-12-05 18:11:34,2021-12-05 18:12:59,Success,NaN,NaN,NaN,Non-CBSA East South Central,True,...,276.80,0.0,1690.0,104.849471,9047.031872,0.0,41.354701,0.062041,5.353273,0.024470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488740,549996,351,2021-12-05 20:27:26,2021-12-05 20:28:43,Success,NaN,NaN,NaN,Non-CBSA Pacific,True,...,203.54,0.0,1690.0,79.204829,27733.918395,0.0,102.552541,0.046867,16.410603,0.060682
488741,549997,312,2021-12-05 20:13:22,2021-12-05 20:14:53,Success,NaN,NaN,NaN,Non-CBSA Mountain,True,...,280.38,0.0,2176.0,1607.740561,1446.949874,0.0,165.711254,0.738851,0.664959,0.076154
488742,549998,273,2021-12-05 21:06:57,2021-12-05 21:07:47,Success,NaN,NaN,NaN,"CBSA Los Angeles-Long Beach-Anaheim, CA",True,...,26.66,0.0,617.0,195.357898,1965.678824,0.0,26.242964,0.316625,3.185865,0.042533
488743,549999,153,2021-12-05 19:33:38,2021-12-05 19:34:32,Success,NaN,NaN,NaN,Non-CBSA East South Central,True,...,104.97,0.0,1220.0,156.384539,2607.570881,0.0,24.535855,0.128184,2.137353,0.020111


### 1.1. Check housing charateristics distributions in BASELINE

In [11]:
proto_chars = ['build_existing_model.geometry_stories',
               'build_existing_model.geometry_wall_type',
               'build_existing_model.vintage_acs',
               'build_existing_model.hvac_cooling_type',
               'build_existing_model.hvac_heating_type_and_fuel',
               'build_existing_model.geometry_floor_area',
               'build_existing_model.census_division',
               'build_existing_model.building_america_climate_zone',
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'>>> ResStock - {pkg_version} - BASELINE result summary:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['building_id'].count()
    Nchar = Nchar/Ns
    Nchar['N_failed'] = len(res[res[char].isnull()])
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(outdir / 'baseline_housing_char_breakdown.csv', index=False)

>>> ResStock - packages LRMF + SF - BASELINE result summary:

  * 488058 / 488745 samples ran successfully, 687 failed, efficacy: 99.9% 

>>> Housing characteristics splits:

  * [1] build_existing_model.geometry_stories
1.0           0.485643
2.0           0.427142
3.0           0.087215
N_failed    687.000000
Name: building_id, dtype: float64

  * [2] build_existing_model.geometry_wall_type
Brick            0.065361
Concrete         0.035020
Steel Frame      0.004778
Wood Frame       0.894840
N_failed       687.000000
Name: building_id, dtype: float64

  * [3] build_existing_model.vintage_acs
1940-59       0.165984
1960-79       0.264497
1980-99       0.270048
2000-09       0.142424
2010s         0.029607
<1940         0.127440
N_failed    687.000000
Name: building_id, dtype: float64

  * [4] build_existing_model.hvac_cooling_type
Central AC      0.540807
Heat Pump       0.106114
None            0.158782
Room AC         0.194297
N_failed      687.000000
Name: building_id, dtype: floa

In [12]:
res[res['completed_status']=="Success"].to_csv(resdir / 'results_up00.csv', index=False)
print('baseline result processed and saved')

baseline result processed and saved


## 2B. PACKAGE summary ###

In [13]:
### upgrade result processing funcs
def assign_utility_rates_to_upgrade(upkg_no, p, ref, HVAC_upgrades_rate_change, for_packages=False):
    
    res = ref.set_index('building_id')[['gas_rate', 'gas_fixed', 'gas_CO2_rate',
                                        'elec_rate', 'elec_fixed', 'elec_CO2_rate'
                                       ]]
    ## assign rates
    p['gas_rate'] = p['building_id'].map(res['gas_rate'])
    p['gas_fixed'] = p['building_id'].map(res['gas_fixed'])
    p['gas_CO2_rate'] = p['building_id'].map(res['gas_CO2_rate'])
    p['elec_rate'] = p['building_id'].map(res['elec_rate'])
    p['elec_fixed'] = p['building_id'].map(res['elec_fixed'])
    p['elec_CO2_rate'] = p['building_id'].map(res['elec_CO2_rate'])
    
    if not for_packages:
        # ind upgrades
        for n in HVAC_upgrades_rate_change.keys():
            new_rates = HVAC_upgrades_rate_change[n]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_rate'] = new_rates[0]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_fixed'] = new_rates[1]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_rate'] = new_rates[2]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_fixed'] = new_rates[3]
    else:
        if isinstance(HVAC_upgrades_rate_change, dict):
            # packages with a dict input
            if upkg_no in HVAC_upgrades_rate_change.keys():
                for m in HVAC_upgrades_rate_change[upkg_no]:
                    idx = (p[(p['apply_upgrade.applicable']==True) &
                           (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                            ]).index
                    p.loc[idx, 'gas_rate'] = NGH_rate
                    p.loc[idx, 'gas_fixed'] = NGH_fixed
                    p.loc[idx, 'elec_rate'] = EH_rate
                    p.loc[idx, 'elec_fixed'] = EH_fixed
        
        else:
            # packages with a list input
            for m in HVAC_upgrades_rate_change:
                idx = (p[(p['apply_upgrade.applicable']==True) &
                       (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                        ]).index
                p.loc[idx, 'gas_rate'] = NGH_rate
                p.loc[idx, 'gas_fixed'] = NGH_fixed
                p.loc[idx, 'elec_rate'] = EH_rate
                p.loc[idx, 'elec_fixed'] = EH_fixed
                
    # assign 0 rates to building with no energy use  
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0 
    
    for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
        p.loc[p['simulation_output_report.applicable']==False, col] = np.nan

    return p

def combine_upgrade_cost_and_lifetime(p):
    
    ## upgrade costs (sum)
    p['upgrade_cost'] = p['simulation_output_report.upgrade_cost_usd']
    cost_cols = list(x for x in p.columns if x.endswith('cost_usd'))
    p = p.drop(cost_cols, axis=1)
    
    ## upgrade lifetime (min)
    lt_cols = list(x for x in p.columns if x.endswith('lifetime_yrs'))
    p['upgrade_lifetime'] = p[lt_cols].min(axis=1)
    p = p.drop(lt_cols, axis=1)

    p['upgrade_cost'] = p['upgrade_cost'].replace([0, None,''],np.nan)
    p['upgrade_lifetime'] = p['upgrade_lifetime'].replace([0, None,''],np.nan)
    
    return p

def get_annual_totals(pp, get_col_only=False):
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
        
    p['ann_gas_cost'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed']
    p['ann_elec_cost'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed']
    p['ann_energy_cost'] = \
        p['ann_gas_cost'] + p['ann_elec_cost']
    
    p['ann_metric_ton_co2e_gas'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_CO2_rate']
    p['ann_metric_ton_co2e_elec'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_CO2_rate']
    p['ann_metric_ton_co2e'] = \
        p['ann_metric_ton_co2e_gas'] + p['ann_metric_ton_co2e_elec']
    
    if get_col_only:
        return p[['ann_gas_cost','ann_elec_cost','ann_energy_cost',
                 'ann_metric_ton_co2e_gas','ann_metric_ton_co2e_elec','ann_metric_ton_co2e']]
    else:
        return p

def get_annual_gas_elec_site_energy_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_therm_gas_saving'] = ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm']
    p['ann_kwh_elec_saving'] = ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh']
    p['ann_mmbtu_site_energy_saving'] = ref['simulation_output_report.total_site_energy_mbtu']-\
        p['simulation_output_report.total_site_energy_mbtu']
    
    if get_col_only:
        return p[['ann_therm_gas_saving','ann_kwh_elec_saving','ann_mbtu_site_energy_saving']]
    else:
        return p
    
def get_thermal_saving_breakdown(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    # gas thermal components:
    p['gas_heating_mmbtu_saving'] = (ref['simulation_output_report.natural_gas_heating_therm']-\
         p['simulation_output_report.natural_gas_heating_therm'])*therm_to_mmbtu
    p['gas_hot_water_mmbtu_saving'] = (ref['simulation_output_report.natural_gas_water_systems_therm']-\
         p['simulation_output_report.natural_gas_water_systems_therm'])*therm_to_mmbtu
    
    # electric thermal components:
    for eu in thermal_eu_main:
        eu_list = list(thermal_eu_df.query('enduse == @eu').index)
        p[f'elec_{eu}_mmbtu_saving'] = (ref[eu_list].sum(axis=1)-p[eu_list].sum(axis=1))*kwh_to_mmbtu
    
    # other thermal components:
    other_heating_eu = [
        'simulation_output_report.fuel_oil_heating_mbtu',
        'simulation_output_report.propane_heating_mbtu',
        'simulation_output_report.wood_heating_mbtu',
    ]
    p['other_heating_mmbtu_saving'] = ref[other_heating_eu].sum(axis=1)-p[other_heating_eu].sum(axis=1)
    
    other_hot_water_eu = [
        'simulation_output_report.fuel_oil_water_systems_mbtu',
        'simulation_output_report.propane_water_systems_mbtu',
    ]
    p['other_hot_water_mmbtu_saving'] = ref[other_hot_water_eu].sum(axis=1)-p[other_hot_water_eu].sum(axis=1)
    
    if get_col_only:
        return p[['gas_heating_mmbtu_saving','gas_hot_water_mmbtu_saving',
                  'elec_heating_mmbtu_saving','elec_cooling_mmbtu_saving','elec_hvac_fan_pump_mmbtu_saving','elec_hot_water_mmbtu_saving',
                  'other_heating_mmbtu_saving','other_hot_water_mmbtu_saving',
                 ]]
    else:
        return p

def get_annual_energy_cost_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """ 
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_gas_cost_saving'] = \
        ref['simulation_output_report.total_site_natural_gas_therm']*ref['gas_rate']+ref['gas_fixed'] - \
        (p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed'])
        
    p['ann_elec_cost_saving'] = \
        ref['simulation_output_report.total_site_electricity_kwh']*ref['elec_rate']+ref['elec_fixed'] - \
        (p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed'])
    
    p['ann_energy_cost_saving'] = p['ann_gas_cost_saving']+p['ann_elec_cost_saving'] #p[['ann_gas_cost_saving','ann_elec_cost_saving']].sum(axis=1)
    
    if get_col_only:
        return p['ann_energy_cost_saving']
    else:
        return p

def get_annual_metric_ton_co2e_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_metric_ton_co2e_saving_gas'] = \
        (ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm'])*p['gas_CO2_rate']
    
    p['ann_metric_ton_co2e_saving_elec'] = \
        (ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh'])*p['elec_CO2_rate']
    
    p['ann_metric_ton_co2e_saving'] = p['ann_metric_ton_co2e_saving_gas'] + p['ann_metric_ton_co2e_saving_elec']
    
    if get_col_only:
        return p['ann_metric_ton_co2e_saving']
    else:
        return p

def print_metrics_report(p, has_comparative_payback=True):
    ### (1) check for simple_payback > 100 yr or if < 0 yr
    if len(p[~p['simple_payback'].isnull()]):
        print(p['simple_payback'].agg(['min','median','max']))
    spb_100 = p[p['simple_payback']>100]
    if len(spb_100)>0:
        print(f'    *PAYBACK1 - too large* upgrade={n} has {len(spb_100)} simple_payback>100 ' +\
              f'(including {len(p[p["simple_payback"]==np.inf])} INF)')
    spb_neg = p[p['simple_payback']<0]
    if len(spb_neg)>0:
        print(f'    *PAYBACK1 - negative*  upgrade={n} has {len(spb_neg)} simple_payback<0 ' +\
              '(due to negative energy cost saving)')

    ### (2) check for comparative_payback > 100 yr or if < 0 yr
    if has_comparative_payback:
        if len(p[~p['comparative_payback'].isnull()]):
            print(p['comparative_payback'].agg(['min','median','max']))
        spb_100 = p[p['comparative_payback']>100]
        if len(spb_100)>0:
            print(f'    *PAYBACK2 - too large* upgrade={n} has {len(spb_100)} comparative_payback>100 ' +\
                  f'(including {len(p[p["comparative_payback"]==np.inf])} INF)')
        spb_neg = p[p['comparative_payback']<0]
        if len(spb_neg)>0:
            print(f'    *PAYBACK2 - negative*  upgrade={n} has {len(spb_neg)} comparative_payback<0 ' +\
                  '(due to negative energy cost saving)')

    ### (3) check for eui==inf
    for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
        eui_inf = p[p[eui]==np.inf]
        if len(eui_inf)>0:
            print(f'       *EUI - inf* upgrade={n} has {len(eui_inf)} {eui}=INF ' +\
                  '(due to fuel introduction from upgrade)')

    ### (4) check for neg carbon savings
    ces_neg = p[p['ann_metric_ton_co2e_saving']<0]
    if len(ces_neg)>0:
        print(f'    *CARBON - negative*  upgrade={n} has {len(ces_neg)} carbon saving<0 ')
        
print('funcs loaded')

funcs loaded


In [14]:
### set utility rates ###
NG_rate_multiplier = 1 # <-----

if NG_rate_multiplier > 1:
    fn_ext = f'_{NG_rate_multiplier}x_gas_prices' # file name extension to add to relevant results
else:
    fn_ext = ''

### utility rates ###########################################
# ref (EIA): 
# avg ComEd res elec rate 2019: $ 0.1330 /kWh
# weighted avg IL gas rate 2019: $ 0.77183 /therm

## electricity ##
# source: residential 2020, https://www.eia.gov/energyexplained/electricity/prices-and-factors-affecting-prices.php
# annual fixed rates = monthly x 12
EH_fixed = 0 * 12 # annual
NEH_fixed = 0 *12 # annual
# avg of summer rates (J,J,A,S) and non-summer rates
EH_rate = 0.1315 # $/kWh, electric rate for electric heating customers
NEH_rate = 0.1315 # $/kWh, electric rate for non-electric heating customers
# marginal carbon emission factor:
# source: eGrid2019, US CO2e: 1,427.8 lb/MWh, https://www.epa.gov/sites/default/files/2021-02/documents/egrid2019_summary_tables.pdf
elec_CO2_rate = 1427.8*0.000453592/1000 # metric tons of CO2e/kWh

## gas ##
# source: US residential 2020, $10.78/Mcf, https://www.eia.gov/dnav/ng/ng_pri_sum_a_EPG0_PRS_DMcf_a.htm
# conversion factors: Mcf -> therm, https://www.eia.gov/tools/faqs/faq.php?id=45&t=8
# annual fixed rates = monthly x 12
GH_fixed = 0 * 12
NGH_fixed = 0 * 12
# variable rates
GH_rate = 10.78/10.37 * NG_rate_multiplier # $/therm, gas rate for NG heating customers
NGH_rate = 10.78/10.37 * NG_rate_multiplier # $/therm, gas rate for non-NG heating customers
# marginal carbon emission factor:
gas_CO2_rate = 0.00532181 # metric tons of CO2e/therm (0.0532 tons/mmbtu), from Elevate's sources

### upgrades that will cause utility rate change: ###########################################
HVAC_upgrades_rate_change = {
    4: [1], # MSHP,
    5: [1], # MSHP,
    6: [1], # MSHP,
}

# assign rates accordingly
Elec_heating_types = ['Electricity Baseboard','Electricity ASHP','Electricity Electric Furnace',
                      'Electricity Electric Boiler', 'Electricity Electric Wall Furnace']
NG_heating_types = ['Natural Gas Fuel Wall/Floor Furnace', 'Natural Gas Fuel Furnace',
                    'Natural Gas Fuel Boiler']

res['gas_rate'] = NGH_rate
res['gas_fixed'] = NGH_fixed
res['gas_CO2_rate'] = gas_CO2_rate
res['elec_rate'] = NEH_rate
res['elec_fixed'] = NEH_fixed
res['elec_CO2_rate'] = elec_CO2_rate

res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
       'gas_rate'] = GH_rate
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
       'gas_fixed'] = GH_fixed
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
        'elec_rate'] = EH_rate
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
        'elec_fixed'] = EH_fixed

# assign 0 gas rates to building with no gas use 
res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0

for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
    res.loc[res['completed_status']!='Success', col] = np.nan

print(f'Natural gas rate multiplier: {NG_rate_multiplier}')


Natural gas rate multiplier: 1


### Get baseline results

In [17]:
#### count upgrades
N_upgrades = 6 # <--

print(f'>>> {pkg_version} has {N_upgrades:,} packages...')
    
## get summary table
summary_upgrades = []

package_list = range(1,1+N_upgrades)
for n in package_list:
    p = pd.read_csv(result_path / f'results_up{n:02d}.csv')
    print(f'\nPackage {n}')
    
    upgrade_name = p['apply_upgrade.upgrade_name'].replace('',np.nan).dropna(axis=0).unique()[0]
    ### assign utility rates
    p = assign_utility_rates_to_upgrade(n, p, res, HVAC_upgrades_rate_change, for_packages=True)
    
    ### collapse upgrade cost and lifetime cols
    p = combine_upgrade_cost_and_lifetime(p)

    ### check if upgrade has 0 successful sims
    if len(p[p['completed_status']=='Success']) == 0:
        print(f' * upgrade={n} has 0 successful simulations')

    ### calculate metrics
    p = add_thermal_load_sqft_eui(p, res)
    EUIi = ['gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_mmbtupersqft']
    EUIo = ['gas_eui','elec_eui','site_eui']
    for vari, varo in zip(EUIi, EUIo):
        p[f'pct_delta_{varo}'] = ((p[vari]-res[vari])/res[vari]*100)


    # annual energy saving:
    p = get_annual_gas_elec_site_energy_saving(p, res)
    p = get_thermal_saving_breakdown(p, res)

    # annual energy cost saving:
    p = get_annual_energy_cost_saving(p, res)
    
    # annual kBtu saved per upgrade cost:
    p['ann_kbtu_saved_per_dollar'] = p['ann_mmbtu_site_energy_saving'].divide(
                            p['upgrade_cost'], axis=0)*1000

    # simple payback
    p['simple_payback'] = p['upgrade_cost']/p['ann_energy_cost_saving']

    # annual metric ton carbon emission savings:
    p = get_annual_metric_ton_co2e_saving(p, res)

    ### check for neg/large paybacks, inf eui, neg carbon savings 
    print_metrics_report(p, has_comparative_payback=False)
    
    ### subset to only those that have been applied with the upgrades successfully 
    # and also have a successful baseline:
    upgrade_name_short = upgrade_rename_dict[upgrade_name]
    p = p[(p['apply_upgrade.applicable']==True) & (res['completed_status']=='Success')].reset_index(drop=True)
    p['package_no'] = n
    

    p['upgrade_name'] = upgrade_name
    p['upgrade_name_short'] = upgrade_name_short
    
    # save processed results
    p[p['completed_status']=='Success'].to_csv(resdir / f'results_up{n:02d}.csv', index=False)
    
    for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
        p[eui] = p[eui].replace([np.inf, -np.inf], np.nan) # for mean calc
        
    ### add to summary table
    summ = pd.DataFrame(index=[0])
    level0 = ''
    summ[(level0, 'upgrade_no')] = n
    summ[(level0, 'upgrade_name')] = upgrade_name
    summ[(level0, 'upgrade_name_short')] = upgrade_name_short
    summ[(level0, 'sample_weight')] = res['build_existing_model.sample_weight'].mean()
    summ[(level0, 'n_applied')] = len(p)
    summ[(level0, 'n_success')] = len(p[p['completed_status']=='Success'])
    summ[(level0, 'n_fail')] = len(p[p['completed_status']=='Fail'])
    summ[(level0, 'n_invalid')] = len(p[p['completed_status']=='Invalid'])
    summ[(level0, 'pct_success')] = round(summ[(level0,'n_success')]/summ[(level0,'n_applied')]*100,3)
    summ.columns = pd.MultiIndex.from_tuples(summ.columns)

    para = ['mean','std', 'min','50%','max']
    p = p[p['completed_status']=='Success'].reset_index(drop=True)
    
    ## (0) Upgrade cost
    summ = pd.concat([
        summ,
        p['upgrade_cost'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    
    ## (1) Baselines
    consum = pd.DataFrame(index=[0])
    level1 = 'baseline - mean'
    # 1.1. Annual Totals
    consum[(level1, 'total gas (therm)')] = round(
        res.loc[p.index, 'simulation_output_report.total_site_natural_gas_therm'].mean(), 
        3)
    consum[(level1, 'total elec (kwh)')] = round(
        res.loc[p.index, 'simulation_output_report.total_site_electricity_kwh'].mean(), 
        3)
    consum[(level1, 'total site (mmbtu)')] = round(
        res.loc[p.index, 'simulation_output_report.total_site_energy_mbtu'].mean(), 
        3)

    # 1.2. Thermal Totals
    consum[(level1, 'thermal gas (therm)')] = round(
        res.loc[p.index, 'thermal_gas_therm'].mean(), 
        3)
    consum[(level1, 'thermal elec (kwh)')] = round(
        res.loc[p.index, 'thermal_elec_kwh'].mean(), 
        3)
    consum[(level1, 'thermal site (mmbtu)')] = round(
        res.loc[p.index, 'thermal_site_mmbtu'].mean(), 
        3)
    
    # 1.3. Thermal EUIs
    consum[(level1, 'thermal_gas_eui (therm/sqft)')] = round(
        res.loc[p.index, 'gas_eui_thermpersqft'].mean(), 
        3)
    consum[(level1, 'thermal_elec_eui (kwh/sqft)')] = round(
        res.loc[p.index, 'elec_eui_kwhpersqft'].mean(), 
        3)
    consum[(level1, 'thermal_site_eui (mmbtu/sqft)')] = round(
        res.loc[p.index, 'site_eui_mmbtupersqft'].mean(), 
        3)
    
    ## (2) Upgrades
    level2 = 'upgrade - mean'
    # 2.1. Annual Totals
    consum[(level2, 'gas (therm)')] = round(p['simulation_output_report.total_site_natural_gas_therm'].mean(), 3)
    consum[(level2, 'elec (kwh)')] = round(p['simulation_output_report.total_site_electricity_kwh'].mean(), 3)
    consum[(level2, 'site (mmbtu)')] = round(p['simulation_output_report.total_site_energy_mbtu'].mean(), 3)
    
    # 2.2. EUIs
    consum[(level2, 'gas_eui (therm/sqft)')] = round(p['gas_eui_thermpersqft'].mean(), 3)
    consum[(level2, 'elec_eui (kwh/sqft)')] = round(p['elec_eui_kwhpersqft'].mean(), 3)
    consum[(level2, 'site_eui (mmbtu/sqft)')] = round(p['site_eui_mmbtupersqft'].mean(), 3)
    
    consum.columns = pd.MultiIndex.from_tuples(consum.columns)
    summ = pd.concat([summ, consum], axis=1)
    
    
    ## (3-1) Savings ##
    level3 = 'savings'
    # 3-1.1 Annual Totals
    summ = pd.concat([
        summ,
        p['ann_therm_gas_saving'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    summ = pd.concat([
        summ,
        p['ann_kwh_elec_saving'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    summ = pd.concat([
        summ,
        p['ann_mmbtu_site_energy_saving'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    
    summ = pd.concat([
        summ,
        p['pct_delta_gas_eui'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    summ = pd.concat([
        summ,
        p['pct_delta_elec_eui'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    summ = pd.concat([
        summ,
        p['pct_delta_site_eui'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    

    
    summary_upgrades.append(summ)
    
summary_upgrades = pd.concat(summary_upgrades).reset_index(drop=True)
summary_upgrades


>>> packages LRMF + SF has 6 packages...

Package 1
min      -1.004606e+15
median    4.522556e+01
max                inf
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=1 has 126150 simple_payback>100 (including 1072 INF)
    *PAYBACK1 - negative*  upgrade=1 has 16714 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=1 has 16589 carbon saving<0 


/opt/anaconda3/envs/EULP-calibration-and-validation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,8,54) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Package 2
min      -4.094774e+17
median    9.088344e+01
max                inf
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=2 has 228902 simple_payback>100 (including 1075 INF)
    *PAYBACK1 - negative*  upgrade=2 has 13785 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=2 has 13698 carbon saving<0 


/opt/anaconda3/envs/EULP-calibration-and-validation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,8,54) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Package 3
min      -3.874400e+07
median    8.527782e+01
max       3.437511e+08
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=3 has 218978 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=3 has 89858 simple_payback<0 (due to negative energy cost saving)
       *EUI - inf* upgrade=3 has 2 pct_delta_gas_eui=INF (due to fuel introduction from upgrade)
    *CARBON - negative*  upgrade=3 has 89073 carbon saving<0 


...

/opt/anaconda3/envs/EULP-calibration-and-validation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,8,54) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Package 5
min      -2.420275e+07
median    5.031281e+01
max       6.474858e+06
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=5 has 103402 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=5 has 36369 simple_payback<0 (due to negative energy cost saving)
       *EUI - inf* upgrade=5 has 8721 pct_delta_elec_eui=INF (due to fuel introduction from upgrade)
       *EUI - inf* upgrade=5 has 92 pct_delta_site_eui=INF (due to fuel introduction from upgrade)
    *CARBON - negative*  upgrade=5 has 35684 carbon saving<0 


/opt/anaconda3/envs/EULP-calibration-and-validation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,8,54) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Package 6
min      -1.081577e+07
median    6.958692e+01
max       4.457262e+07
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=6 has 166665 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=6 has 45627 simple_payback<0 (due to negative energy cost saving)
       *EUI - inf* upgrade=6 has 8720 pct_delta_elec_eui=INF (due to fuel introduction from upgrade)
       *EUI - inf* upgrade=6 has 92 pct_delta_site_eui=INF (due to fuel introduction from upgrade)
    *CARBON - negative*  upgrade=6 has 44706 carbon saving<0 


\
  upgrade_no                                     upgrade_name   
0          1                                Airtight Envelope   
1          2                          Extra-Airtight Envelope   
2          3                Ultra-Airtight Envelope - HRV/ERV   
3          4                  Airtight Envelope + MSHP + HPWH   
4          5            Extra-Airtight Envelope + MSHP + HPWH   
5          6  Ultra-Airtight Envelope - HRV/ERV + MSHP + HPWH   

                                                                              \
       upgrade_name_short sample_weight n_applied n_success n_fail n_invalid   
0                     Env    272.477584    488049    487458      0       591   
1                ExtraEnv    272.477584    488043    488043      0         0   
2                UltraEnv    272.477584    487961    487961      0         0   
3       Env + MSHP + HPWH    272.477584    488028    488027      0         1   
4  ExtraEnv + MSHP + HPWH    272.477584    488020    488020      0         0   
5  UltraEnv + MSHP + HPWH    272.477584    487981    487981      0         0   

              upgrade_cost  ... pct_delta_elec_eui                        \
  pct_success         mean  ...               mean           std     min   
0      99.879     12636.81  ...             -20.08  1.947000e+01 -100.00   
1     100.000     36079.27  ...             -26.25  2.308000e+01 -100.00   
2     100.000     52027.75  ...             -26.43  2.995000e+01 -100.00   
3     100.000     20140.31  ...          387625.21  2.179186e+08  -95.17   
4     100.000     43214.87  ...          390680.48  2.205087e+08  -95.92   
5     100.000     58760.06  ...          401407.13  2.236416e+08  -96.86   

                       pct_delta_site_eui                                  
     50%           max               mean    std     min    50%       max  
0 -16.96  4.482300e+02             -27.22  17.02  -98.01 -25.85    260.59  
1 -23.91  1.295500e+03             -36.58  18.69 -100.00 -37.62    374.04  
2 -23.74  2.543080e+03             -43.97  21.79 -100.00 -46.48   1387.05  
3 -28.10  1.504934e+11             -74.31  46.65  -97.45 -82.15  16245.54  
4 -36.04  1.523003e+11             -77.43  39.39  -97.60 -84.35  13647.00  
5 -40.56  1.543820e+11             -79.36  31.52  -98.23 -85.88  10299.33  

[6 rows x 59 columns]

In [18]:
### export UPGRADE summary
summary_upgrades.to_csv(outdir / f'upgrades_summary.csv', index=False)
print(f'UPGRADE summary table saved')

UPGRADE summary table saved
